In [ ]:
# clean all prs beta file

import pandas as pd
import os
from multiprocessing import Pool, cpu_count

beta_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/beta/"
output_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/beta/"
# TRAIT_LIST = ['waist', 'height', 'pulse', 'dbp', 'sbp', 'smoke', 'drink', 'bmi', 'wbc', 'rbc', 'hb', 'plt', 'lymph', 'mono', 'neut', 'eos', 'alt', 'ast', 'bun', 'cholesterol', 'creatinine', 'glucose', 'ggt', 'hdl', 'ldl', 'triglycerides', 'ua']
TRAIT_LIST = ['ua', 'triglycerides', 'glucose', 'hdl']

def process_single_file(beta_file_path, output_path):
    temp_beta = pd.read_csv(beta_file_path, sep="\t", header=0)
    temp_beta["new_id"] = temp_beta.apply(lambda row: f'{row["SNP"].split(":")[0]}' if str(row["SNP"]).startswith("rs") else f'{row["SNP"]}', axis=1)
    res_beta = temp_beta[['new_id', 'A1', 'BETA']]
    res_beta.columns = ['SNP', 'A1', 'BETA']
    res_beta.to_csv(output_path, sep="\t", index=False, header=True)

for trait in TRAIT_LIST:
    print(f"Processing trait: {trait}")
    for group_num in range(1, 11):
        beta_file_path = os.path.join(beta_base_dir, f"{trait}/group_{group_num}/final_betas.txt")
        if not os.path.exists(beta_file_path):
            print(f"Beta file does not exist: {beta_file_path}")
            continue
        output_prefix = os.path.join(output_base_dir, f"{trait}/group_{group_num}/cleaned_betas.txt")
        process_single_file(beta_file_path, output_prefix)
        print(f"Processed: {beta_file_path} -> {output_prefix}")

In [ ]:
# command = f'{plink_path} --bfile {test_bfile_path} --score {beta_file} 1 2 3 header --out {output_path}/prs_{p}_{h}_{sample_size_EUR_label[j]}_{sample_size_EAS_label[i]}_P{m}'

import pandas as pd
import os
from multiprocessing import Pool, cpu_count

beta_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/beta/"
test_bfile_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/geno/Chinese/1_merged/merged"
plink_path = "/data1/jiapl_group/lishuhua/software/general/plink"
output_base_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/test_prs/"
# TRAIT_LIST = ['waist', 'height', 'pulse', 'dbp', 'sbp', 'smoke', 'drink', 'bmi', 'wbc', 'rbc', 'hb', 'plt', 'lymph', 'mono', 'neut', 'eos', 'alt', 'ast', 'bun', 'cholesterol', 'creatinine', 'glucose', 'ggt', 'hdl', 'ldl', 'triglycerides', 'ua']
TRAIT_LIST = ['ua', 'triglycerides', 'glucose', 'hdl']

def process_single_file(beta_file_path, output_path):
    cmd = f"{plink_path} --bfile {test_bfile_path} --score {beta_file_path} 1 2 3 header no-mean-imputation --out {output_path}"
    os.system(cmd)

for trait in TRAIT_LIST:
    print(f"Processing trait: {trait}")
    for group_num in range(1, 11):
        beta_file_path = os.path.join(beta_base_dir, f"{trait}/group_{group_num}/cleaned_betas.txt")
        if not os.path.exists(beta_file_path):
            print(f"Beta file does not exist: {beta_file_path}")
            continue
        output_prefix = os.path.join(output_base_dir, f"{trait}/group_{group_num}")
        if not os.path.exists(os.path.dirname(output_prefix)):
            os.makedirs(os.path.dirname(output_prefix))
        process_single_file(beta_file_path, output_prefix)
        print(f"Processed: {beta_file_path} -> {output_prefix}")

In [ ]:
import pandas as pd
import os
import sys

prs_file_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/test_prs/"
output_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/test_prs_cleaned"
TRAIT_LIST = ['waist', 'height', 'pulse', 'dbp', 'sbp', 'smoke', 'drink', 'bmi', 'wbc', 'rbc', 'hb', 'plt', 'lymph', 'mono', 'neut', 'eos', 'alt', 'ast', 'bun', 'cholesterol', 'creatinine', 'glucose', 'ggt', 'hdl', 'ldl', 'triglycerides', 'ua']

for trait in TRAIT_LIST:
    print(f"Processing trait: {trait}")
    for group_num in range(1, 11):
        prs_file = os.path.join(prs_file_path, f"{trait}/group_{group_num}.profile")
        if not os.path.exists(prs_file):
            print(f"PRS file does not exist: {prs_file}")
            continue
        prs_data = open(prs_file, "r")
        prs_list = []
        for line in prs_data:
            line_arr = line.strip().split(" ")
            line_arr = [x for x in line_arr if x != ""]
            prs_list.append(line_arr)
        prs_data.close()
        prs_df = pd.DataFrame(prs_list[1:], columns=prs_list[0])
        trait_name = f"{trait}_group_{group_num}"
        prs_df.to_csv(os.path.join(output_path, f"{trait_name}_PRS.tsv"), sep='\t', index=False, header=True)

In [ ]:
# /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/test_prs_cleaned/{triat}_group_{group_num}_PRS.tsv
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, average_precision_score, mean_squared_error
from scipy.stats import pearsonr
import os
import warnings

# Ignore warnings that may arise from certain fits in statsmodels
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

# --- 1. Metric Calculation Functions ---

def calculate_continuous_metrics(df, base_covars, full_covars):
    """Calculates all performance metrics for a continuous trait for a given dataset (df)."""
    # Incremental R²
    model_base = sm.OLS(df["trait"], sm.add_constant(df[base_covars])).fit()
    model_full = sm.OLS(df["trait"], sm.add_constant(df[full_covars])).fit()
    r2_incremental = model_full.rsquared - model_base.rsquared

    # Pearson correlation coefficient (SCORE vs. phenotype residuals)
    pheno_residuals = model_base.resid
    corr, _ = pearsonr(df["SCORE"], pheno_residuals)

    # RMSE
    prediction_full = model_full.predict(sm.add_constant(df[full_covars]))
    rmse = np.sqrt(mean_squared_error(df["trait"], prediction_full))
    nrmse_mean = rmse / df["trait"].mean() if df["trait"].mean() != 0 else np.nan
    nrmse_range = rmse / (df["trait"].max() - df["trait"].min()) if (df["trait"].max() - df["trait"].min()) != 0 else np.nan
    nrmse_std = rmse / df["trait"].std() if df["trait"].std() != 0 else np.nan

    # Quantile means
    df['quantile'] = pd.qcut(df['SCORE'], 5, labels=False, duplicates='drop')
    quantile_means = df.groupby('quantile')['trait'].mean()
    
    return {
        "r2_incremental": r2_incremental,
        "r2_full": model_full.rsquared,
        "rmse": rmse,
        "nrmse_mean": nrmse_mean,
        "nrmse_range": nrmse_range,
        "nrmse_std": nrmse_std,
        "pearson_r": corr,
        "top_quintile_mean": quantile_means.iloc[-1] if not quantile_means.empty else np.nan,
        "bottom_quintile_mean": quantile_means.iloc[0] if not quantile_means.empty else np.nan
    }

def calculate_binary_metrics(df, base_covars, full_covars):
    """Calculates all performance metrics for a binary trait for a given dataset (df)."""
    # AUC and PR-AUC
    logit_model = sm.Logit(df["trait"], sm.add_constant(df[full_covars])).fit(disp=0)
    pred_prob = logit_model.predict(sm.add_constant(df[full_covars]))
    auc = roc_auc_score(df["trait"], pred_prob)
    pr_auc = average_precision_score(df["trait"], pred_prob)

    # OR per 1-SD
    df["prs_scaled"] = (df["SCORE"] - df["SCORE"].mean()) / df["SCORE"].std()
    logit_model_scaled = sm.Logit(df["trait"], sm.add_constant(df[base_covars + ["prs_scaled"]])).fit(disp=0)
    or_per_sd = np.exp(logit_model_scaled.params["prs_scaled"])

    # Quantile OR
    df['prs_quintile'] = pd.qcut(df['SCORE'], 5, labels=False, duplicates='drop')
    reference_quintile = 2 # Middle quintile
    or_quintiles = {}
    for q in range(5):
        if q == reference_quintile:
            or_quintiles[f'OR_Quintile_{q+1}'] = 1.0
            continue
        
        # Check if both current and reference quintiles exist in the data
        if not df['prs_quintile'].isin([q, reference_quintile]).all():
             or_quintiles[f'OR_Quintile_{q+1}'] = np.nan
             continue
             
        temp_df = df[df['prs_quintile'].isin([q, reference_quintile])].copy()
        
        # Check for sufficient data in both groups for stable model fitting
        if temp_df['trait'].nunique() < 2 or temp_df['prs_quintile'].nunique() < 2:
            or_quintiles[f'OR_Quintile_{q+1}'] = np.nan
            continue
            
        temp_df['is_current_quintile'] = (temp_df['prs_quintile'] == q).astype(int)
        X_quintile = sm.add_constant(temp_df[['is_current_quintile'] + base_covars])
        try:
            model_q = sm.Logit(temp_df["trait"], X_quintile).fit(disp=0)
            or_quintiles[f'OR_Quintile_{q+1}'] = np.exp(model_q.params['is_current_quintile'])
        except Exception:
            or_quintiles[f'OR_Quintile_{q+1}'] = np.nan
            
    results = {
        "auc": auc,
        "pr_auc": pr_auc,
        "or_per_sd": or_per_sd,
    }
    results.update(or_quintiles) # Merge quantile ORs into the results dictionary
    return results

# --- 2. Main Execution Flow ---

def main():
    # --- Parameter Settings ---
    # !! Note: Please modify the variables below according to your actual paths !!
    cleaned_prs_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/test_prs_cleaned/"
    covar_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_chinese_final.tsv"
    output_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/ctsleb/res/full_res/"
    pheno_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/trait/Chinese/"
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # --- Initialization ---
    final_results_continuous = []
    final_results_binary = []
    covar_cols = ["FID", "IID", "age", "sex"] + [f"PC{i}" for i in range(1, 11)]
    base_covars = ["age", "sex"] + [f"PC{i}" for i in range(1, 11)]
    full_covars = base_covars + ["SCORE"]

    # --- Data Loading and Processing ---
    covars = pd.read_csv(covar_path, sep='\t', usecols=covar_cols)

    trait_dict = {
        'p48': 'waist',
        'p50': 'height',
        'p102': 'pulse',
        'p4079': 'dbp',
        'p4080': 'sbp',
        'p20116': 'smoke',
        'p20117': 'drink',
        'p21001': 'bmi',
        'p30000': 'wbc',
        'p30010': 'rbc',
        'p30020':'hb',
        'p30080': 'plt',
        'p30120': 'lymph',
        'p30130': 'mono',
        'p30140': 'neut',
        'p30150': 'eos',
        'p30620': 'alt',
        'p30650': 'ast',
        'p30670': 'bun',
        'p30690': 'cholesterol',
        'p30700': 'creatinine',
        'p30730': 'ggt',
        'p30740': 'glucose',
        'p30760': 'hdl',
        'p30780': 'ldl',
        'p30870': 'triglycerides',
        'p30880': 'ua'
    }

    for trait, name in trait_dict.items():
        for i in range(1, 11):
            trait_prs_path = os.path.join(cleaned_prs_path, f"{name}_group_{i}_PRS.tsv")
            print(f"Loading PRS file: {trait_prs_path}")
            if not os.path.exists(trait_prs_path):
                print(f"Warning: PRS file for trait {name} group {i} not found. Skipping.")
                continue
            print(f"Processing Trait: {name}, Group: {i}")
            if trait == 'p20116' or trait == 'p20117':
                trait_id_from_prs = f"{trait}_int"
            else:
                trait_id_from_prs = f"{trait}_raw"
            pheno_path = os.path.join(pheno_dir, f"{trait_id_from_prs}.txt")
            pheno = pd.read_csv(pheno_path, sep='\t', header=None)
            pheno.columns = ["FID", "IID", "trait"]
            prs = pd.read_csv(trait_prs_path, sep='\t')

            for df in [pheno, prs, covars]:
                df["FID"] = df["FID"].astype(str)
                df["IID"] = df["IID"].astype(str)
            merged_data = pd.merge(pheno, prs, on=["FID", "IID"], how="inner")
            merged_data = pd.merge(merged_data, covars, on=["FID", "IID"], how="inner")

            # Defensive data cleaning and type conversion
            numeric_cols = ["trait", "SCORE", "age", "sex"] + [f"PC{i}" for i in range(1, 11)]
            for col in numeric_cols:
                if col in merged_data.columns:
                    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')
            
            original_rows = len(merged_data)
            merged_data.dropna(subset=numeric_cols, inplace=True)
            new_rows = len(merged_data)
            if original_rows > new_rows:
                print(f"--> Warning: Dropped {original_rows - new_rows} rows due to non-numeric data or NaNs in trait {trait_id_from_prs}.")
            if new_rows == 0:
                print(f"--> Error: No valid samples remained for trait {trait_id_from_prs} after cleaning. Skipping.")
                continue
            print(f"Data merged and cleaned for trait {trait_id_from_prs}. Total samples: {len(merged_data)}")

            if trait_id_from_prs in ["p20116_int", "p20117_int"]:
                # Binary trait analysis
                # Ensure binary trait is 0/1 coded
                unique_vals = sorted(merged_data["trait"].unique())
                if not set(unique_vals).issubset({0, 1}):
                    if len(unique_vals) == 2:
                        print(f"Converting binary trait from {unique_vals} to 0/1.")
                        merged_data["trait"] = (merged_data["trait"] == unique_vals[1]).astype(int)
                    else:
                        print(f"Error: Binary trait column for {trait_id_from_prs} contains unexpected values: {unique_vals}. Skipping.")
                        continue
                
                # Direct calculation of metrics
                analysis_report = calculate_binary_metrics(merged_data, base_covars, full_covars)
                analysis_report['trait'] = trait_id_from_prs
                final_results_binary.append(analysis_report)
            else:
                # Continuous trait analysis
                # Direct calculation of metrics
                analysis_report = calculate_continuous_metrics(merged_data, base_covars, full_covars)
                analysis_report['trait'] = trait_id_from_prs
                final_results_continuous.append(analysis_report)

    # --- 3. Save Final Results ---
    if final_results_continuous:
        continuous_df = pd.DataFrame(final_results_continuous)
        # Reorder columns to have 'trait' first
        cols = ['trait'] + [col for col in continuous_df.columns if col != 'trait']
        continuous_df = continuous_df[cols]
        continuous_df.to_csv(os.path.join(output_dir, "prs_continuous_metrics.csv"), index=False)
        print("\nContinuous trait results saved to prs_continuous_metrics.csv")
        print(continuous_df)

    if final_results_binary:
        binary_df = pd.DataFrame(final_results_binary)
        # Reorder columns to have 'trait' first
        cols = ['trait'] + [col for col in binary_df.columns if col != 'trait']
        binary_df = binary_df[cols]
        binary_df.to_csv(os.path.join(output_dir, "prs_binary_metrics.csv"), index=False)
        print("\nBinary trait results saved to prs_binary_metrics.csv")
        print(binary_df)

if __name__ == '__main__':
    main()